In [2]:
import pandas as pd
import numpy as np
import json
import re
from nltk.corpus import stopwords
import os

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_json("Data_cyb.json", lines = True, orient = "columns")

In [4]:
rating = []

for i in df["annotation"]:
    rating.append(i["label"][0])
    
df["rating"] = rating

In [5]:
new_df = df[["content", "rating"]]
new_df.tail()

,content,rating
19996,I dont. But what is complaining about it goi...,0
19997,Bahah yeah i&;m totally just gonna&; get pis...,0
19998,hahahahaha >:) im evil mwahahahahahahahahaha,0
19999,What&;s something unique about Ohio? :),0
20000,Who is the biggest gossiper you know?,0


In [6]:
new_df.to_csv("comments.csv")

In [7]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20001 entries, 0 to 20000
Data columns (total 2 columns):
content    20001 non-null object
rating     20001 non-null object
dtypes: object(2)
memory usage: 156.3+ KB


In [8]:
X, X_test, y, y_test = train_test_split(new_df["content"], new_df["rating"], train_size = 0.8)


In [9]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(X)
reviews_test_clean = preprocess_reviews(X_test)

In [10]:
reviews_train_clean[5]

'i know what u mean  my im hsdpa sucks as well'

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

In [12]:
len(reviews_train_clean), len(reviews_test_clean)

(16000, 4001)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))


c:\users\nary tang\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy for C=0.01: 0.67875
Accuracy for C=0.05: 0.72125
Accuracy for C=0.25: 0.774
Accuracy for C=0.5: 0.7965
Accuracy for C=1: 0.8155


In [15]:
final_model = LogisticRegression(C=0.05)
final_model.fit(X, y)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, final_model.predict(X_test)))

Final Accuracy: 0.7343164208947763


In [16]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print (best_positive)
    

    
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print (best_negative)
    

('fuck', 1.9298262185378487)
('gay', 1.7094774897679044)
('bitch', 1.6066418579898725)
('emo', 1.5279301705296076)
('fucking', 1.4319414069286858)
('ever', -0.8403730789330629)
('favorite', -0.49785220639423544)
('nope', -0.49191452562280835)
('im', -0.4656844581225645)
('love', -0.4530094461829309)
